In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import urllib

In [ ]:
# url="http://www.samaratrans.info/wiki/index.php/%D0%A1%D0%B0%D0%BC%D0%B0%D1%80%D0%B0_%D0%B0%D0%B2%D1%82%D0%BE%D0%B1%D1%83%D1%81_1"


In [383]:
def getTable(table_rows):
    l = []
    for tr in table_rows:
        try:
            td = tr.find_all('td')
            row = [tr.text for tr in td]
            l.append(row)
        except:
            pass
    #
    return l

In [523]:
def end(city,tp_ts,region,nmbr,rt_nm,route_frw,route_bckw,stops_frw,stops_bckw,nlst,url):
    lst_all = [city,tp_ts,region,nmbr,rt_nm,route_frw,route_bckw,stops_frw,stops_bckw]
    
    def foo(nlst,i):
        return nlst[i][1]

    for i in range(9):
        try: 
            tmp = foo(nlst,i)
        except:
            tmp = None
        lst_all.append(tmp)
    # 
    lst_all.append(url)
    return lst_all

In [524]:
def getInfo(url,grp_name,city):
    req = requests.get(url)
    bs = BeautifulSoup(req.text, 'html.parser')
    
    region = grp_name
    #     region='муниципальный'
    str_url = checkEnd(url)
    
    lst = str_url.split("_")
    # 2 - номер маршрута
    nmbr = lst[-1]
    # 1 - тип транспорта
    tp_ts = lst[-2]
    # rt_nmbr = url.rsplit("_",1)[1]
    
    # 2 - содержание
    # 3 - маршрут следования
    # 5 - оп в прямом направлении
    # 6 - оп в обратном направлении
    # 7 - особенности работы
    # 8 - расписание (две версии, для регионов!!)

    #####

    all_tables = bs.find_all("table")

    #######
    l2,l3,l4,l5,l6,l7,l8 = [],[],[],[],[],[],[]
    for i in range(len(all_tables)):
        l = getTable(all_tables[i])
        if 'Содержание'in l[0][0]:
            l2 = l
        elif ('направление'in l[0][0]) & (len(l[0][0]) < 40):
            try: 
                tmp = l[1][0]
            except:
                tmp = ''
            if ('прям' in l[0][0]) | ('прям' in tmp):
                if '№п' not in tmp:
                    l3 = l
                else:
                    l5 = l
            elif ('обратн' in l[0][0]) | ('обратн' in tmp):
                if '№п' in tmp:
                    l6 = l
            #
        elif ('обратное'in l[0][0]) & (len(l[0][0]) < 40):
            try: 
                tmp = l[1][0]
            except:
                tmp = ''
            if '№п' in tmp:
                l6 = l
        elif 'работы'in l[0][0]:
            l7 = l
        else:
            l8 = l
    # 

    ####
    # 3 - наименование маршрута
    str1 = l2[0][0]
    str1 = str1.replace("\n\n","\n")
    rt_nm = str1.split("\n")[1][2:].split("маршрут ")[1]
    rt_nm = rt_nm.split('"', 1)[1][:-1]

    ####
    # 4 -маршрут следования
    route_frw = l3[0][1]
    try:
        route_bckw = l3[1][1]
    except:
        route_bckw = 'кольцевой'
    ####

    ####
    # 5,6 - список оп
    def stops(l):
    #
        str_all=''
        for row in l[2:]:
            rw = []
            for i in row:
                i = i.replace("\n","")
                rw.append(i)
            str1 = rw[1]
            str_all = str_all + "_" + str1
        #
        str_all = str_all[1:]
        return str_all
    # 
    stops_frw = stops(l5)
    try:
        stops_bckw = stops(l6)
    except:
        stops_bckw = 'кольцевой'
    ####

    ####
    # 7) Режим работы
    # 8) Интервалы движения
    # 9) Выпуск на линию
    # 10) Время в пути
    # 11) Протяжённость маршрута
    # 12) Обслуживающие предприятия
    # 13) Подвижной состав
    # 14) Стоимость проезда
    # 15) Место отстоя 

    nlst = []
    for row in l7:
        tmplst = []
        for j in row:
            j = j.replace("\n","")
            tmplst.append(j)
        nlst.append(tmplst)
    # 
    lst_all = end(city,tp_ts,region,nmbr,rt_nm,route_frw,route_bckw,stops_frw,stops_bckw,nlst,url)
    
#     lst_all = [city,tp_ts,region,nmbr,rt_nm,route_frw,route_bckw,stops_frw,stops_bckw]
#     #                     work_time,intervs,amnt_mch,duration,length,orgnztn,podv_sost,payment,place_rest,url]
    
#     def foo(nlst,i):
#         return nlst[i][1]

#     for i in range(9):
#         try: 
#             tmp = foo(nlst,i)
#         except:
#             tmp = None
#         lst_all.append(tmp)
#     # 
#     lst_all.append(url)

    # 
    return lst_all

In [525]:
def getInfo2(url,grp_name,city):
    # Сызрань спешл
    req = requests.get(url)
    bs = BeautifulSoup(req.text, 'html.parser')
    
    region = grp_name
    #     region='муниципальный'
    str_url = checkEnd(url)
    
    lst = str_url.split("_")
    # 2 - номер маршрута
    nmbr = lst[-1]
    # 1 - тип транспорта
    tp_ts = lst[-2]
    # rt_nmbr = url.rsplit("_",1)[1]

    all_tables = bs.find_all("table")
    
        # 3 - наименование маршрута
    rt_nm = str(bs.find_all('b')[2]).replace("<b>","").replace("</b>","")

    def getLines(lst_s,nmb):
        lst_ok = []

        for i in range(len(lst_s)):
            lst1 = lst_s[i][nmb].split("\n")
            for s in lst1:
                if s != '':
                    lst_ok.append(s)
        # 
        return lst_ok

    def stops(lst_s,nmb):
        lst_ok = getLines(lst_s,nmb)

        str1=''
        for row in lst_ok:
            if row != '\n':
                row = row.replace("\n", "")
                str1 = str1 + "_" + row
        # 
        str1 = str1[1:]
        return str1

    ####
    
    i=0
    for i in range(len(all_tables)):
        l = getTable(all_tables[i])
        if "в сторону" in l[0][0]:
            l5 = getTable(all_tables[i+2])
            l6 = getTable(all_tables[i+3])
            break
        else:
            l5 = getTable(all_tables[5])
            l6 = getTable(all_tables[6])
    # 

    ####
    # 4 -маршрут следования

    route_frw = stops(l5,0)
    try:
        route_bckw = stops(l6,0)
    except:
        route_bckw = 'кольцевой'
    ####

    # 5,6 - список оп
    stops_frw = stops(l5,1)
    try:
        stops_bckw = stops(l6,1)
    except:
        stops_bckw = 'кольцевой'
    ####
    
    nlst = []
    lst_all = end(city,tp_ts,region,nmbr,rt_nm,route_frw,route_bckw,stops_frw,stops_bckw,nlst,url)
    
#     lst_all = [city,tp_ts,region,nmbr,rt_nm,route_frw,route_bckw,stops_frw,stops_bckw]
#     #                     work_time,intervs,amnt_mch,duration,length,orgnztn,podv_sost,payment,place_rest,url]
    
#     def foo(nlst,i):
#         return nlst[i][1]

#     for i in range(9):
#         try: 
#             tmp = foo(nlst,i)
#         except:
#             tmp = None
#         lst_all.append(tmp)
#     # 
#     lst_all.append(url)
    
    return lst_all

In [2]:
def checkGrpNm(tmp_grp_nm):
    if tmp_grp_nm == 'маршруты':
        grp_name = 'муниципальный'
    elif  tmp_grp_nm == 'коммерческий_маршруты':
        grp_name = 'коммерческий'
    elif  tmp_grp_nm == 'пригородный_расписания':
        grp_name = 'пригородный'
    elif  tmp_grp_nm == 'сезонный_маршруты':
        grp_name = 'сезонный'
    elif  tmp_grp_nm == 'междугородный_расписания':
        grp_name = 'междугородний'
    else:
        grp_name = tmp_grp_nm
    # 
    return grp_name

In [3]:
def checkEnd(url):
    the_string = url.rsplit('php/',1)[1]
    srch = re.search('[а-яА-Я]', the_string) 
    if srch == None:
        str_url = urllib.parse.unquote(the_string)
    else:
        str_url = the_string
    #
    return str_url

In [4]:
# 
def linksCity(city):

    url_grp = 'http://www.samaratrans.info/wiki/index.php/' + city
    req = requests.get(url_grp)
    bs = BeautifulSoup(req.text, 'html.parser')
    stfa = bs.find_all('a')
    links=[]
    for link in stfa:
        try:
            if 'маршрут' in link.get("title"):
                lsh = 'http://www.samaratrans.info' + link.get("href")
                if lsh not in links:
                    links.append(lsh)
        except:
            pass
    # 
    return links

In [21]:
def routeLinks(url_grp):
    links=[]
    req = requests.get(url_grp)
    bs = BeautifulSoup(req.text, 'html.parser')
    stfa = bs.find_all('table')
    
    cnt=2
    for i in range(len(stfa)):
        table_rows = stfa[i]
        l = getTable(table_rows)
        if 'Наименование маршрута\n' in l[0]:
            cnt = i
            break
    # 
    lst_find = stfa[cnt].find_all('a')
    for link in lst_find:
        try:
            lst_notok = ['расписан', 'парк', '.gif', '.png', 'перевозчик', 'edit', 'ТД', 'ТТУ']
            lnk_hr = link.get("href")
            ch = checkEnd(lnk_hr)
            if not any((c in ch) for c in lst_notok):
                lsh = 'http://www.samaratrans.info' + lnk_hr
                if lsh not in links:
                    links.append(lsh)
        except:
            pass
    # 
    return links

In [349]:
def getCityData(city):
    lst_links = []
    lst_links = linksCity(city)
    all_routes=[]
    for grp in range(len(lst_links)):
        tmp_grp_nm = checkEnd(lst_links[grp]).split("_",2)[-1]
        grp_name = checkGrpNm(tmp_grp_nm)
        
        try:
            lst_group = []
            lst_group = routeLinks(lst_links[grp])
        except:
            lst_group = []
        for rtl in range(len(lst_group)):
            try:
                tmp_lnk_nm = checkEnd(lst_group[rtl])
                lst_notok = ['расписан', 'парк', '.gif', '.png', 'перевозчик', 'edit', 'ТД', 'ТТУ']
                if not any((c in tmp_lnk_nm) for c in lst_notok):
                    try:
                        one_route = getInfo(lst_group[rtl],grp_name,city)
                    except:
                        one_route = getInfo2(lst_group[rtl],grp_name,city)
                    all_routes.append(one_route)
                #
            except:
                one_route = [lst_group[rtl]] * 19
                if one_route not in all_routes:
                    all_routes.append(one_route)
                #
    return all_routes
# 

In [350]:
lst_ct = ['Самара', 'Тольятти', 'Сызрань', 'Новокуйбышевск', 
          'Чапаевск', 'Жигулевск', 'Отрадный', 'Кинель', 'Похвистнево', 'Безенчук']

In [377]:
lst_big = []
for i in range(len(lst_ct)):
    city = lst_ct[i]
    print(city)
    lst = getCityData(city)
    lst_big = lst_big + lst
# 

Самара
Тольятти
Сызрань
Новокуйбышевск
Чапаевск
Жигулевск
Отрадный
Кинель
Похвистнево
Безенчук


In [352]:
clmn = []
for i in range(1,len(lst_big[0])+1):
    str1 = str(i)+'str'
    clmn.append(str1)
# 

In [378]:
df_big = pd.DataFrame(data=lst_big,columns=clmn)

In [379]:
print(len(df_big))

447


In [380]:
lst_not_ok = list(set(list(df_big[df_big["1str"].str.contains("www")]["1str"])))

In [381]:
lst2 = list(set(lst_not_ok))

In [382]:
print(len(lst2))

0


In [359]:
for i in lst2:
    print(i)

http://www.samaratrans.info/wiki/index.php/%D0%A1%D1%8B%D0%B7%D1%80%D0%B0%D0%BD%D1%8C_%D0%B0%D0%B2%D1%82%D0%BE%D0%B1%D1%83%D1%81_7
http://www.samaratrans.info/wiki/index.php/%D0%A1%D1%8B%D0%B7%D1%80%D0%B0%D0%BD%D1%8C_%D0%B0%D0%B2%D1%82%D0%BE%D0%B1%D1%83%D1%81_6
http://www.samaratrans.info/wiki/index.php/%D0%A1%D1%8B%D0%B7%D1%80%D0%B0%D0%BD%D1%8C_%D0%B0%D0%B2%D1%82%D0%BE%D0%B1%D1%83%D1%81_8
http://www.samaratrans.info/wiki/index.php/%D0%A1%D1%8B%D0%B7%D1%80%D0%B0%D0%BD%D1%8C_%D0%B0%D0%B2%D1%82%D0%BE%D0%B1%D1%83%D1%81_116
http://www.samaratrans.info/wiki/index.php/%D0%A1%D1%8B%D0%B7%D1%80%D0%B0%D0%BD%D1%8C_%D0%B0%D0%B2%D1%82%D0%BE%D0%B1%D1%83%D1%81_3
http://www.samaratrans.info/wiki/index.php/%D0%A1%D1%8B%D0%B7%D1%80%D0%B0%D0%BD%D1%8C_%D0%B0%D0%B2%D1%82%D0%BE%D0%B1%D1%83%D1%81_35
http://www.samaratrans.info/wiki/index.php/%D0%A1%D1%8B%D0%B7%D1%80%D0%B0%D0%BD%D1%8C_%D0%B0%D0%B2%D1%82%D0%BE%D0%B1%D1%83%D1%81_2
http://www.samaratrans.info/wiki/index.php/%D0%A1%D1%8B%D0%B7%D1%80%D0%B0%D0%BD%

In [360]:
url = lst2[0]
grp_name = 'error'
city = 'Сызрань'
print(url)

http://www.samaratrans.info/wiki/index.php/%D0%A1%D1%8B%D0%B7%D1%80%D0%B0%D0%BD%D1%8C_%D0%B0%D0%B2%D1%82%D0%BE%D0%B1%D1%83%D1%81_7


In [386]:
ltp=getInfo2(url,grp_name,city)

In [526]:
url_ot = "http://www.samaratrans.info/wiki/index.php/%D0%A0%D0%B0%D0%B9%D0%BE%D0%BD%D1%8B_%D0%B0%D0%B2%D1%82%D0%BE%D0%B1%D1%83%D1%81_%D1%80%D0%B0%D1%81%D0%BF%D0%B8%D1%81%D0%B0%D0%BD%D0%B8%D1%8F"

In [527]:
def getOtherLinks(url_ot):
    req = requests.get(url_ot)
    bs = BeautifulSoup(req.text, 'html.parser')

    all_lnks = bs.find_all('a')
    lst_rasp=[]
    i=0
    for link in all_lnks:
        if "расписани" in str(link.get('title')):
            lsh = "http://www.samaratrans.info" + link.get('href')
            lst_rasp.append(lsh)
    # 

    return lst_rasp

In [528]:
lst_rasp = getOtherLinks(url_ot)

In [555]:
def getInfo3(url):

    req = requests.get(url)
    bs = BeautifulSoup(req.text, 'html.parser')

    city = checkEnd(url).split("_",1)[0]




    ####
    # get stops
    # stops_frw
    i=0
    lst_stops = []
    all_p = bs.find_all('p')
    for p in all_p:
        if "Промежуточные" in str(p):
            try:
                txt = p.text.replace("Промежуточные остановки: ","").replace("\n","")
                lst_stops.append(txt)
            except:
                pass
    # 
    ####
    ####
    i=0
    all_tables = bs.find_all('table')
    for i in range(len(all_tables)):
        l = getTable(bs.find_all('table')[0])
        if "Расписание" in l[0][0]:
            l1 = l
            break
        else:
            l1 = getTable(bs.find_all('table')[0])
    # 
    lst_nms = l1[0][0].split("\n")
    ####

    ####
    lst_reg = []
    lst_tpts = []
    lst_rtnbr = []
    lst_rtnm = []
    for i in range(len(lst_nms)):
        str1 = lst_nms[i]
        if "Расписание" in str1:
            if " муниципальн" in str1:
                region = "муниципальный"
            elif " межмуниципальн" in str1:
                region = "межмуниципальный"
            else:
                region = "другой"
        else:
            if "маршрут" in str1:
    #             lst_rt.append(str1)
                tp_ts, rt = str1.split(" ",1)[1].split("№ ",1)
                rt_nbr, rt_nm = rt.split('"',1)
                rt_nbr = rt_nbr.replace(" ", "")
                rt_nm = rt_nm[:-1]

                if "втобус" in tp_ts:
                    tp_ts = "автобус"
                elif "рамвай" in tp_ts:
                    tp_ts = "трамвай"
                elif "роллейбус" in tp_ts:
                    tp_ts = "троллейбус"
                else:
                    tp_ts = "другой"
                # 
                lst_rtnbr.append(rt_nbr)
                lst_rtnm.append(rt_nm)
                lst_tpts.append(tp_ts)

                lst_reg.append(region)
    # 
    ####

    ####
    lst_all = []
    for i in range(len(lst_stops)):
        tp_ts = lst_tpts[i]
        region = lst_reg[i]
        nmbr = lst_rtnbr[i]
        rt_nm = lst_rtnm[i]
        route_frw = None
        route_bckw = None
        stops_frw = lst_stops[i]
        stops_bckw = None
        nlst = []
        small_lst_all = end(city,tp_ts,region,nmbr,rt_nm,route_frw,route_bckw,stops_frw,stops_bckw,nlst,url)
        lst_all.append(small_lst_all)
    # 
    ####

    return lst_all

In [556]:
lst_big2 = []
i=0
for i in tqdm(range(len(lst_rasp))):
    url = lst_rasp[i]
    try:
        lst = getInfo3(url)
    except:
        lst = [[url] * 19]
    #
    for j in range(len(lst)):
        lst_big2.append(lst[j])
#     lst_big2 = lst_big2 + lst
# 

In [532]:
# from tqdm.notebook import tqdm

In [557]:
df_othr = pd.DataFrame(data=lst_big2,columns=clmn)

In [558]:
df_othr

,1str,2str,3str,4str,5str,6str,7str,8str,9str,10str,11str,12str,13str,14str,15str,16str,17str,18str,19str
0,Алексеевка,автобус,муниципальный,1,с. Алексеевка - с. Несмеяновка,None,None,нет,None,None,None,None,None,None,None,None,None,None,http://www.samaratrans.info/wiki/index.php/%D0...
1,Алексеевка,автобус,муниципальный,2,с. Алексеевка - с. Самовольно-Ивановка,None,None,"с. Калашиновка (пов.), с/х им. Титова (п. Суха...",None,None,None,None,None,None,None,None,None,None,http://www.samaratrans.info/wiki/index.php/%D0...
2,Алексеевка,автобус,муниципальный,3,с. Алексеевка - п. Дальний,None,None,"с. Несмеяновка (пов.), с/х ""Авангард"" (п. Аван...",None,None,None,None,None,None,None,None,None,None,http://www.samaratrans.info/wiki/index.php/%D0...
3,Алексеевка,автобус,муниципальный,4,с. Алексеевка - п. Субботинский,None,None,нет,None,None,None,None,None,None,None,None,None,None,http://www.samaratrans.info/wiki/index.php/%D0...
4,Алексеевка,автобус,муниципальный,5,с. Алексеевка - п. Шариповка,None,None,"с. Новотроевка, с. Патровка, с. Гавриловка, п....",None,None,None,None,None,None,None,None,None,None,http://www.samaratrans.info/wiki/index.php/%D0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,Шигоны,автобус,межмуниципальный,584,г. Сызрань (Ж/д вокзал) - п. Волжский Утёс,None,None,"с. Троекуровка, с/х ""Пионер"" (п. Пионерский, п...",None,None,None,None,None,None,None,None,None,None,http://www.samaratrans.info/wiki/index.php/%D0...
153,Шигоны,автобус,межмуниципальный,620,г. Сызрань (Ж/д вокзал) - с. Усолье,None,None,"с. Троекуровка, с/х ""Пионер"" (п. Пионерский, п...",None,None,None,None,None,None,None,None,None,None,http://www.samaratrans.info/wiki/index.php/%D0...
154,Шигоны,автобус,межмуниципальный,568/568д,г. Самара (ЦАВ) - с. Шигоны / с. Усолье,None,None,"КДП ""Новосемейкино"", Поворот на Самару (СНТ ""С...",None,None,None,None,None,None,None,None,None,None,http://www.samaratrans.info/wiki/index.php/%D0...
155,Шигоны,автобус,межмуниципальный,668,г. Тольятти (Новый Город) - с. Усолье,None,None,"г. Тольятти (ТЦ ""Аврора"", Старый Город, п. Жиг...",None,None,None,None,None,None,None,None,None,None,http://www.samaratrans.info/wiki/index.php/%D0...


In [560]:
df_othr[df_othr["1str"].str.contains("www")]

,1str,2str,3str,4str,5str,6str,7str,8str,9str,10str,11str,12str,13str,14str,15str,16str,17str,18str,19str
23,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...
36,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...
37,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...
56,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/wiki/index.php/%D0...
66,http://www.samaratrans.info/wiki/index.php/%D0...,http://www.samaratrans.info/w